In [1]:
import os
from PIL import Image
from tqdm.notebook import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration

In [2]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

2024-04-21 11:16:56.792904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 11:16:56.793047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 11:16:56.919646: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
def generate_caption(images_path, out_path):
    animals = [folder for folder in os.listdir(images_path) if not folder.endswith('.txt')]

    for animal in tqdm(animals):
        image_caption_dict = {}
        animal_dir = os.path.join(images_path, animal)
        image_files = [f for f in os.listdir(animal_dir)]
        
        for image_file in tqdm(image_files):
            image_path = os.path.join(animal_dir, image_file)
            raw_image = Image.open(image_path).convert('RGB')
            
            inputs = processor(raw_image, return_tensors="pt").to('cuda')
            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)
            
            # Store image path and its corresponding caption in the dictionary
            image_caption_dict[image_file] = caption
            
            # Write the image paths and their corresponding captions to a text file
        with open(out_path+animal+'.txt', 'w') as file:
            for image_file, caption in image_caption_dict.items():
                file.write(f"{image_file},{caption}\n")

In [4]:
generate_caption('/kaggle/input/coco-animal-dataset', '/kaggle/working/')

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/2782 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/4117 [00:00<?, ?it/s]

  0%|          | 0/926 [00:00<?, ?it/s]

...

  0%|          | 0/1316 [00:00<?, ?it/s]

  0%|          | 0/1724 [00:00<?, ?it/s]

  0%|          | 0/1616 [00:00<?, ?it/s]

  0%|          | 0/2187 [00:00<?, ?it/s]

  0%|          | 0/4041 [00:00<?, ?it/s]